In [2]:
pip install requests pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install schedule

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import requests
import time
import schedule
from datetime import datetime

In [5]:
API_KEY = "fc515aa87d309d82df472bd53f53048d"

In [ ]:
def load_districts(file_path="Ind_adm2_Points.csv"):
 
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='ISO-8859-1')

    df_clean = df[["State", "District", "Latitude", "Longitude"]]
    
    df_clean = df_clean.dropna(subset=["District", "Latitude", "Longitude"])
    
    df_unique = df_clean.drop_duplicates(subset="District", keep='first')

    return df_unique


In [30]:
def fetch_forecast(lat, lon):
    url = f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API_KEY}&units=metric"
    res = requests.get(url)
    return res.json()['list']

In [31]:
def summarize_forecast(forecast):
    temps = [entry['main']['temp'] for entry in forecast]
    hums = [entry['main']['humidity'] for entry in forecast]
    rains = [entry.get('rain', {}).get('3h', 0.0) for entry in forecast]

    return {
        "temperature": round(sum(temps) / len(temps), 2),
        "humidity": round(sum(hums) / len(hums), 2),
        "rainfall": round(sum(rains), 2)
    }

In [32]:
def fetch_all_weather():
    print("🌤 Fetching forecast for all districts...")
    districts_df = load_districts()
    all_data = []

    for _, row in districts_df.iterrows():
        state = row['State']
        district = row['District']
        lat = row['Latitude']
        lon = row['Longitude']

        try:
            forecast = fetch_forecast(lat, lon)
            summary = summarize_forecast(forecast)

            all_data.append({
                "date": datetime.now().strftime("%Y-%m-%d"),
                "state": state,
                "district": district,
                **summary
            })
            print(f"✅ {district} ({state}): {summary}")
        except Exception as e:
            print(f"❌ Failed for {district}: {e}")

    df_out = pd.DataFrame(all_data)
    filename = f"weather_{datetime.now().strftime('%Y-%m-%d')}.csv"
    df_out.to_csv(filename, index=False)
    print(f"\n📁 Weather saved to: {filename}\n")


In [33]:
fetch_all_weather()

🌤 Fetching forecast for all districts...
✅ Andaman Islands (Andaman and Nicobar): {'temperature': 27.83, 'humidity': 81.05, 'rainfall': 49.46}
✅ Nicobar Islands (Andaman and Nicobar): {'temperature': 27.35, 'humidity': 83.08, 'rainfall': 263.83}
✅ Srikakulam (Andhra Pradesh): {'temperature': 30.1, 'humidity': 71.53, 'rainfall': 3.92}
✅ Vishakhapatnam (Andhra Pradesh): {'temperature': 30.07, 'humidity': 72.12, 'rainfall': 1.71}
✅ Vizianagaram (Andhra Pradesh): {'temperature': 27.75, 'humidity': 73.53, 'rainfall': 17.35}
✅ Warangal (Andhra Pradesh): {'temperature': 29.92, 'humidity': 68.45, 'rainfall': 4.57}
✅ West Godavari (Andhra Pradesh): {'temperature': 30.4, 'humidity': 67.92, 'rainfall': 10.94}
✅ East Godavari (Andhra Pradesh): {'temperature': 31.96, 'humidity': 58.98, 'rainfall': 1.48}
✅ Guntur (Andhra Pradesh): {'temperature': 31.69, 'humidity': 61.4, 'rainfall': 2.91}
✅ Krishna (Andhra Pradesh): {'temperature': 31.18, 'humidity': 65.4, 'rainfall': 2.5}
✅ Nellore (Andhra Pradesh)